## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease       
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease         
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(6, 7):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:23<00:00,  3.09it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_6"])
layer_hs_array.shape

(10000, 267264)

## Layer 6 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

# dim_reduced_vecs = layer_hs_array

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 7894)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 7894D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 0.04 s
Outer iteration 0 / 10
  Iteration 19 (0.77 s, search 0.54 s): objective=5191.38 imbalance=1.527 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (1.53 s, search 1.08 s): objective=5217.07 imbalance=1.268 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (2.30 s, search 1.61 s): objective=5215.05 imbalance=1.277 nsplit=0       
Outer iteration 3 / 10
  Iteration 19 (3.06 s, search 2.15 s): objective=5198.87 imbalance=1.251 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (3.83 s, search 2.69 s): objective=5189.25 imbalance=1.217 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (4.59 s, search 3.22 s): objective=5209.39 imbalance=1.252 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (5.36 s, search 3.76 s): objective=5189.16 imbalance=1.492 nsplit=0       
Outer iteration 7 / 10
  Iter

5217.0712890625

In [10]:
kmeans.centroids #cluster centers

array([[ 0.00828712, -0.01291786,  0.01888424, ..., -0.00114952,
         0.00774072, -0.00312124],
       [ 0.01092559, -0.01258994,  0.00541091, ..., -0.00927354,
         0.00052226,  0.00132347],
       [ 0.01036584, -0.01887489,  0.01206024, ..., -0.00359276,
         0.00555541,  0.00736123],
       ...,
       [ 0.00818718, -0.01529382,  0.0126172 , ..., -0.00789918,
        -0.00353992,  0.00850426],
       [ 0.00760146, -0.01904487,  0.01231206, ..., -0.01271863,
         0.00913911,  0.00335464],
       [ 0.00988952, -0.01330086,  0.00585044, ..., -0.01388716,
        -0.00513776,  0.00462051]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

0.99999976
1.0000004
1.0000001
1.0000001
1.0000002
0.99999994
1.0000002
0.99999994
0.9999999
1.0000001


In [12]:
kmeans.obj #inertia at each iteration

array([3087.61987305, 5022.83007812, 5132.73046875, 5166.48242188,
       5173.64648438, 5180.46484375, 5184.60253906, 5186.51855469,
       5187.67724609, 5188.45947266, 5189.04443359, 5189.48632812,
       5189.86474609, 5190.18164062, 5190.41796875, 5190.63183594,
       5190.93115234, 5191.15283203, 5191.26123047, 5191.37988281,
       3216.21459961, 4996.05859375, 5142.1171875 , 5181.31201172,
       5197.12158203, 5206.43603516, 5211.87695312, 5214.51171875,
       5215.47021484, 5215.97265625, 5216.35498047, 5216.62792969,
       5216.83349609, 5216.95800781, 5217.01464844, 5217.02685547,
       5217.05078125, 5217.06054688, 5217.06591797, 5217.07128906])

In [13]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

6    1905
1    1774
5    1449
3    1150
4     916
8     786
0     556
9     548
2     504
7     412
dtype: int64

In [14]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [15]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [16]:
pd.Series(classifications).value_counts()

6    1905
1    1774
5    1449
3    1150
4     916
8     786
0     556
9     548
2     504
7     412
dtype: int64